In [1]:
# pip install langchain_community

In [2]:
from langchain_community.llms import Ollama

In [3]:
llm = Ollama(model="llama3.2")
llm.invoke("The first man on the moon was")

/var/folders/vl/9bmd_w5x5r5f6pvxvgqmj16c0000gn/T/ipykernel_13937/2823046338.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


'Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, during the Apollo 11 mission.'

# Reading transaction data

In [4]:
# Reading the transection data
import pandas as pd
df = pd.read_csv("My_Transection.csv")

In [5]:
df

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08
7,2023-12-22,Consulting,Income,541.57
8,2023-12-22,Aidsfonds,Expense,10.70
9,2023-12-20,Consulting,Income,2641.93


In [6]:
# Getting unique transactions in the Name / Description column
unique_transactions = df["Expense/Income"].unique()
len(unique_transactions)

2

In [7]:
# Getting unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [8]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

# Categorise bank transactions with Llama2

In [9]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [10]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [11]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam', llm)

['Here are the categories for each expense:', '', '* Spotify AB by Adyen - Entertainment', '* Beta Boulders Ams Amsterdam Nld - Sport', '* ISS Catering Services De Meern - Food', '* Taxi Utrecht - Transportation', '* Etos AMSTERDAM NLD - Shopping', '* Bistro Bar Amsterdam - Food', '', "Let me know if you'd like me to adjust any of these categories!"]


,Transaction vs category,Transaction,Category
0,Here are the categories for each expense:,Here are the categories for each expense:,None
1,,,None
2,* Spotify AB by Adyen - Entertainment,* Spotify AB by Adyen,Entertainment
3,* Beta Boulders Ams Amsterdam Nld - Sport,* Beta Boulders Ams Amsterdam Nld,Sport
4,* ISS Catering Services De Meern - Food,* ISS Catering Services De Meern,Food
5,* Taxi Utrecht - Transportation,* Taxi Utrecht,Transportation
6,* Etos AMSTERDAM NLD - Shopping,* Etos AMSTERDAM NLD,Shopping
7,* Bistro Bar Amsterdam - Food,* Bistro Bar Amsterdam,Food
8,,,None
9,Let me know if you'd like me to adjust any of ...,Let me know if you'd like me to adjust any of ...,None


In [12]:
# Initialize an empty DataFrame to store all results
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list) - 1):
    # Slice the unique_transactions list to get transaction names in the current range
    transaction_names = unique_transactions[index_list[i]:index_list[i + 1]]
    transaction_names = ','.join(transaction_names)

    # Call the categorize_transactions function
    categories_df = categorize_transactions(transaction_names, llm)

    # Check if 'Transaction vs category' column exists in the returned DataFrame
    if 'Transaction vs category' in categories_df:
        # Identify rows that do not contain ' - ' to split correctly
        missing_delimiters = ~categories_df['Transaction vs category'].str.contains(' - ')

        # Handle rows with missing delimiters by providing a placeholder or dropping them
        categories_df['Transaction vs category'] = categories_df['Transaction vs category'].where(
            ~missing_delimiters, 'Unknown - Unknown'
        )

        # Split 'Transaction vs category' column into 'Transaction' and 'Category' columns
        categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    else:
        raise ValueError("Column 'Transaction vs category' is missing from the response")

    # Concatenate the current DataFrame with the overall results DataFrame
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

# Now categories_df_all should contain all categorized transactions with 'Transaction' and 'Category' columns

['Here are the added categories:', '', '1. Belastingdienst - Taxes', '2. Tesco Breda - Grocery', '3. Monthly Apartment Rent - Housing', '4. Vishandel Sier Amsterdam - Furniture', '5. Selling Paintings - Art', '6. Spotify Ab By Adyen - Entertainment', '7. Tk Maxx Amsterdam Da - Shopping', '8. Consulting - Services', '9. Aidsfonds - Charity', '10. Tls Bv Inz Ov-Chipkaart - Travel', '11. Etos Amsterdam - Clothing', '12. Beta Boulders Ams Amsterdam - Sport', '13. Salary - Income', '14. Bouldermuur Bv Amsterdam - Hobby', '15. Birtat Restaurant Amsterdam - Dining', '16. Freelancing - Income', '17. Tikkie - Expenses', '18. Blogging - Hobbies', '19. Taxi Utrecht - Transportation', '20. Apple Services - Technology', '21. Amazon Lux - Shopping', '22. Classpass* Monthly - Fitness', '23. Audible Uk AdblCo/Pymt Gbr - Subscriptions']


In [13]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Unknown - Unknown,Unknown,Unknown
1,Unknown - Unknown,Unknown,Unknown
2,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
3,2. Tesco Breda - Grocery,2. Tesco Breda,Grocery
4,3. Monthly Apartment Rent - Housing,3. Monthly Apartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Furniture,4. Vishandel Sier Amsterdam,Furniture
6,5. Selling Paintings - Art,5. Selling Paintings,Art
7,6. Spotify Ab By Adyen - Entertainment,6. Spotify Ab By Adyen,Entertainment
8,7. Tk Maxx Amsterdam Da - Shopping,7. Tk Maxx Amsterdam Da,Shopping
9,8. Consulting - Services,8. Consulting,Services


In [14]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Unknown', 'Taxes', 'Grocery', 'Housing', 'Furniture', 'Art',
       'Entertainment', 'Shopping', 'Services', 'Charity', 'Travel',
       'Clothing', 'Sport', 'Income', 'Hobby', 'Dining', 'Expenses',
       'Hobbies', 'Transportation', 'Technology', 'Fitness',
       'Subscriptions'], dtype=object)

In [15]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [16]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

,Transaction vs category,Transaction,Category
0,Unknown - Unknown,Unknown,Unknown
1,Unknown - Unknown,Unknown,Unknown
2,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
3,2. Tesco Breda - Grocery,2. Tesco Breda,Grocery
4,3. Monthly Apartment Rent - Housing,3. Monthly Apartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Furniture,4. Vishandel Sier Amsterdam,Furniture
6,5. Selling Paintings - Art,5. Selling Paintings,Art
7,6. Spotify Ab By Adyen - Entertainment,6. Spotify Ab By Adyen,Entertainment
8,7. Tk Maxx Amsterdam Da - Shopping,7. Tk Maxx Amsterdam Da,Shopping
9,8. Consulting - Services,8. Consulting,Services


In [17]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("My_Transection.csv")
df.loc[df['Name / Description'].str.contains("Apple"), 'Name / Description'] = "Apple Services"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,NaN,NaN,NaN
1,2023-12-30,Tesco Breda,Expense,17.53,NaN,NaN,NaN
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN,NaN,NaN
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN,NaN,NaN
4,2023-12-29,Selling Paintings,Income,13.63,NaN,NaN,NaN
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,2023-12-22,Consulting,Income,541.57,NaN,NaN,NaN
8,2023-12-22,Aidsfonds,Expense,10.70,NaN,NaN,NaN
9,2023-12-20,Consulting,Income,2641.93,NaN,NaN,NaN


In [18]:
df.to_csv("new_transection.csv", index=False)